# Graphical Interfaces: Plotly Dash

Several frameworks are available to build applications around database systems.
Popular frameworks for web applications in Python are Flask, Streamlit, and Plotly Dash.

We will use Plotly Dash in this course.

Cover this quick tutorial: https://dash.plotly.com/tutorial and the explore the full features of this rich framework. 

You can edit and lauch your app [directly from a jupyter notebook](https://dash.plotly.com/jupyter-notebooks/).

In [ ]:
pip install --user dash

In [ ]:
# override the container login credentials 
import os
import getpass

os.environ['DJ_HOST'] = 'percona-qa.datajoint.io'   # used in class in Fall 2025
os.environ['DJ_USER'] = getpass.getpass('Username:')
os.environ['DJ_PASS'] = getpass.getpass('Password:')

In [ ]:
import datajoint as dj
dj.list_schemas()

In [ ]:
schema = dj.Schema('dimitri-yatsenko_db-course_gallery')

In [ ]:
@schema
class ImageSource(dj.Lookup):
    definition = """
        image_name :  varchar(20)
        ---
        url  : varchar(255)
        """
    
    contents = (
        ("SPECT", r"https://upload.wikimedia.org/wikipedia/commons/1/1a/SPECT_Slice_of_Brain_using_Tc-99m_Ceretec.jpg"),
        ("cytoskeleton", r"https://upload.wikimedia.org/wikipedia/commons/1/12/Neuron_actin_cytoskeleton.JPG"),
        ("axon", r"https://upload.wikimedia.org/wikipedia/commons/6/6c/Axons_from_an_Alzheimer%E2%80%99s_mouse_model.jpg"),
        )

In [ ]:
ImageSource()

In [ ]:
import urllib.request
import matplotlib.pyplot as plt
import PIL.Image
import numpy as np

@schema
class Image(dj.Imported):
    definition = """
    -> ImageSource
    ---
    image : longblob
    """

    def make(self, key):
        url = (ImageSource & key).fetch1('url')
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(url, 'temp.jpg')
        try:
            image = PIL.Image.open("temp.jpg")
            self.insert1(dict(key, image=np.array(image)))
        finally:
            # Clean up temporary file
            if os.path.exists('temp.jpg'):
                os.remove('temp.jpg')
        


In [ ]:
Image.populate(display_progress=True)

In [ ]:
Image * ImageSource

In [ ]:
plt.imshow((Image & {'image_name':"axon"}).fetch1('image'))
plt.axis(False)

In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px


# Initialize the app
app = Dash()

# App layout
app.layout = [
    html.Div(children='Simple Database App'),
    html.Hr(),
    dash_table.DataTable(data=ImageSource.fetch(as_dict=True), page_size=6),
    html.Hr(),
    dcc.RadioItems(options=ImageSource.fetch('image_name'), value='axon', id='image-selector'),
    dcc.Graph(id='image-graph')
]
 
@callback(
    Output('image-graph', 'figure'),
    Input('image-selector', 'value')
)
def update_image(selection):
    img = (Image & {'image_name': selection}).fetch1('image')
    return px.imshow(img)

In [ ]:
# Run the app
app.run(jupyter_mode="external")

## VS Code / Cursor Port Forwarding

If you're using VS Code or Cursor with a remote connection:
- Open the Ports panel (View → Ports or Ctrl+Shift+P → "Forward a Port")
- Click "Forward a Port" and enter 8050
- Click on the forwarded URL that appears